<a href="https://colab.research.google.com/github/denisabrantesredis/denisd-GenAI-Workshop/blob/main/Labs/02-LLM/02_Redis_Gemini_LLM.ipynb" target="_newt">
<img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

<div style="display:flex;width=100%;">
<img src="https://redis.io/wp-content/uploads/2024/04/Logotype.svg?auto=webp&quality=85,75&width=120" alt="Redis" width="90"/>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src="https://www.gstatic.com/devrel-devsite/prod/v0e0f589edd85502a40d78d7d0825db8ea5ef3b99ab4070381ee86977c9168730/cloud/images/cloud-logo.svg" alt="Google Cloud" width="140"/>
</div>

# LLM Memory with Redis & Google Cloud

<img src="https://github.com/denisabrantesredis/denisd-GenAI-Workshop/blob/main/_assets/images/redis_gcp.png?raw=true" alt="Redis and Google Cloud" align="center"/>

In this exercise, we will create a conversational chat bot using Google Gemini. We will use Redis to score the conversation history to provide the model with the necessary context.

## Installing the Pre-Reqs

In [ ]:
!pip install -q redis==5.0.8 >> /.tmp
!pip install -q langchain==0.3.25 >> /.tmp
!pip install -q langchain-core==0.3.59 >> /.tmp
!pip install -q langchain-redis==0.2.0 >> /.tmp
!pip install -q langchain-google-genai==2.1.4 >> /.tmp

## Installing Redis Stack Locally
If you are not using Redis Cloud as a database, uncomment and run the code below to install Redis locally. Then set your connection to 127.0.0.1

In [ ]:
# %%sh
# curl -fsSL https://packages.redis.io/gpg | sudo gpg --dearmor -o /usr/share/keyrings/redis-archive-keyring.gpg 
# echo "deb [signed-by=/usr/share/keyrings/redis-archive-keyring.gpg] https://packages.redis.io/deb $(lsb_release -cs) main" | sudo tee /etc/apt/sources.list.d/redis.list 
# sudo apt-get update  > /dev/null 2>&1
# sudo apt-get install redis-stack-server  > /dev/null 2>&1
# redis-stack-server --daemonize yes

## Part 1 - Load and Configure the Model

In this lab, we will use the [Unstructured](https://docs.unstructured.io/open-source/core-functionality/partitioning#partition-html) API to load data from a web page, parse it and break into chunks.

A web page can have multiple different types of content; this class will help us identify the type of content being collected from the page, so we can make sure we're only getting the text from the page.

In [ ]:
import os
import redis
from google.colab import userdata

from langchain_redis import RedisChatMessageHistory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_google_genai import ChatGoogleGenerativeAI

from IPython.display import Markdown

### Step 1: Setting Up Connection String

<img src="https://github.com/denisabrantesredis/denisd-GenAI-Workshop/blob/main/_assets/images/callout_secrets.png?raw=true" alt="Callout - Use Google Colab secrets instead"/>

In [ ]:
try:
  os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')
except:
  os.environ["GOOGLE_API_KEY"] = "<insert API key here>"

try:
  REDIS_HOST = userdata.get('REDIS_HOST')
except:
  REDIS_HOST="127.0.0.1"

try:
  REDIS_PORT = userdata.get('REDIS_PORT')
except:
  REDIS_PORT=6379

try:
  REDIS_PASSWORD = userdata.get('REDIS_PASSWORD')
except:
  REDIS_PASSWORD=""

REDIS_URL = f"redis://default:{REDIS_PASSWORD}@{REDIS_HOST}:{REDIS_PORT}"

#### Testing the Connection to Redis

<img src="https://github.com/denisabrantesredis/denisd-GenAI-Workshop/blob/main/_assets/images/callout_connection.png?raw=true" alt="Callout - Make sure connection works"/>

In [ ]:
r = redis.from_url(REDIS_URL)

if r.ping():
    print("Connection successful!")
else:
    print("Connection issue!")

&nbsp;

### Step 2 - Load the Model

A session identifier is needed to separate conversations per user (or session). We'll use the same Langchain package from the previous labs to orchestrate the conversation with the model. For this use case, we will use `RedisChatMessageHistory`to keep track of the conversation history between the user and the model.

<img src="https://github.com/denisabrantesredis/denisd-GenAI-Workshop/blob/main/_assets/images/callout_session2.png?raw=true" alt="Callout - Session Identifier"/>

In [ ]:
user_session = "default"

Load the model

In [ ]:
llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-pro",
    temperature=0.5,
    top_p=0.95,
    top_k=64,
    max_output_tokens=8192
    )

For this lab, we will keep the prompt very simple, and ensure that the conversation history is included with every interaction.

In [ ]:
# Create a conversational chain
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful AI assistant."),
    MessagesPlaceholder(variable_name="history"),
    ("human", "{input}")
])
chain = prompt | llm

Using Redis to store conversation history is very simple; the storing and retrieval of conversation history is automatically handled by Langchain. For this lab, we're setting a Time-To-Live (TTL) of 3,600 seconds, or 1 hour.

In [ ]:
# Function to get or create a RedisChatMessageHistory instance
def get_redis_history(session_id: str):
    return RedisChatMessageHistory(session_id, redis_url=REDIS_URL, ttl=3600)

In [ ]:
# Create a runnable with message history
chain_with_history = RunnableWithMessageHistory(
    chain,
    get_redis_history,
    input_messages_key="input",
    history_messages_key="history"
)

<img src="https://github.com/denisabrantesredis/denisd-GenAI-Workshop/blob/main/_assets/images/callout_llmfunction.png?raw=true" alt="Callout - LLM Function"/>

In [ ]:
def generate_response(input_text, user_session):
    response = chain_with_history.invoke({"input": input_text}, config={"configurable": {"session_id": user_session}})
    return response.content

## Part 2 - Talking to the Model

Large Language Models do not keep track of the conversations they are having with users; each question is received (and answered) in a completely isolated way, with no context of previous interactions. However, behavior like this would make for very poor interactions, as users need to contextualize what they are trying to achieve, often across multiple interactions.

In order to provide a better user experience, client UIs (like chatbots) often keep track of the previous interactions between users and models, and send this context to the model with every new question. That way, the model can 'read' the previous interactions and contextualize its response in a way that emulate a natural conversation between people.

<img src="https://github.com/denisabrantesredis/denisd-GenAI-Workshop/blob/main/_assets/images/callout_1q.png?raw=true" alt="Callout - First Question to the Model"/>

In [ ]:
chat_input = "What is the capital of Canada?"

Send the question to the model and print the response

In [ ]:
response = generate_response(chat_input, user_session)

In [ ]:
Markdown(response)

&nbsp;

<img src="https://github.com/denisabrantesredis/denisd-GenAI-Workshop/blob/main/_assets/images/callout_insight.png?raw=true" alt="Callout - Check Redis Insight"/>

Open Redis Insight and you will find 2 new documents there, one for the question and one for the answeer.

Notice the additional metadata, such as session identifier, timestamp, etc.

&nbsp;

&nbsp;

### Asking a follow-up session

The Langchain integration greatly simplifies the process of running a semantic search. A single function call is enough. Notice how we do not need to generate a vector for our question manually; this is handled automatically by the function, based on the embedding model we've selected before.

For more details on the different ways to run vector searches, check the [Langchain documentation page](https://python.langchain.com/docs/integrations/vectorstores/redis/#query-vector-store).

&nbsp;


<img src="https://github.com/denisabrantesredis/denisd-GenAI-Workshop/blob/main/_assets/images/callout_2q.png?raw=true" alt="Callout - Follow-up Question to the Model"/>

In [ ]:
chat_input = "How come it's not Toronto?"

### Sending the follow-up question to the model

Without the conversation history, the model would be confused with this new question, seeing as it would be missing the context from the previous interaction.

In [ ]:
response = generate_response(chat_input, user_session)

In [ ]:
Markdown(response)

&nbsp;

<img src="https://github.com/denisabrantesredis/denisd-GenAI-Workshop/blob/main/_assets/images/callout_insight.png?raw=true" alt="Callout - Check Redis Insight"/>

Here's an interesting exercise: open Redis Insight and delete all documents. This will cause the model to get confused when we ask another follow-up question.

<img src="https://github.com/denisabrantesredis/denisd-GenAI-Workshop/blob/main/_assets/images/callout_3q.png?raw=true" alt="Callout - Another Follow-up Question to the Model"/>

In [ ]:
chat_input = "What other cities would be good candidates?"

In [ ]:
response = generate_response(chat_input, user_session)

In [ ]:
Markdown(response)

&nbsp;

## Part 3: Behind the Scenes

In this lab, we will use the Gemini Pro 1.5 model from Google to generate a response to the user, based on the documents retrieved from Redis. The GCP API Key that we set before is required to allow access to the model.

#### Looking at the Message History

In [ ]:
get_redis_history(user_session).messages

#### Querying Redis Directly

Since the conversation history is stored in Redis as documents, we can access them directly for other use cases (for instance, a call center that wants to review the interaction between user and model, or a data sciences team gathering data to fine-tune a conversational model).

Run a Search for documents that contain the word 'city'. Because of [stemming](https://redis.io/docs/latest/develop/interact/search-and-query/advanced-concepts/stemming/), the response will include question and answer

In [ ]:
def run_search(keyword):
  try:
    result = r.ft("idx:chat_history").search(f'@content:{keyword}').docs
  except Exception as e:
    result = f"Error: {e}"

  return result

In [ ]:
results = run_search("city")
for result in results:
  print(result.json)

### Render conversation history in HTML format

In [ ]:
import json
from datetime import datetime
from IPython.display import HTML

In [ ]:
messages = []
for result in results:
  jsondoc = json.loads(result.json)
  msg_from = jsondoc["type"]
  msg_content = jsondoc["data"]["content"]
  ts = jsondoc['timestamp']
  msg_ts = datetime.utcfromtimestamp(ts).strftime('%Y-%m-%d %H:%M:%S')
  
  message = {
      "from" : msg_from,
      "timestamp" : msg_ts,
      "content" : msg_content
  }

  messages.append(message)

In [ ]:
html_start = """
<html><body>
<table style='border:1px solid gray;padding:2px;float:left;'>
    <tr>
      <th style='text-align:center;'>From</th>
      <th style='text-align:center;'>Timestamp</th>
      <th style='text-align:center;'>Content</th>
  </tr>
"""
html_body = ""
for message in messages:
  html_body += f"""
    <tr style='border:1px solid gray;padding:2px;'>
        <td style='text-align:center;width:10%;'>{message['from']}</td>
        <td style='text-align:center;width:20%;'>{message['timestamp']}</td>
        <td style='text-align:left;width:70%;'>{message['content']}</td>
    </tr>
  """
html_end = """
</table></body></html>
"""
html_full = html_start + html_body + html_end

In [ ]:
HTML(html_full)

&nbsp;


&nbsp;



### Important

Redis can be used as Vector Database, Semantic Cache and LLM Memory in the same use case; This allows for much faster interactions and an overall better user experience.

<img src="https://github.com/denisabrantesredis/denisd-GenAI-Workshop/blob/main/_assets/images/diagram_redis_cache_llm.png?raw=true" alt="Diagram - Redis as Semantic Cache and LLM Memory"/>

&nbsp;

# Congrats, this is the end of the lab!!